## Step 2

Traducción a Python de **step_02_train_tumor_normal_classifier_for_deployment.m**

Comentarios:

- En el código original se usaba la resolución de la red de MatLab (224x224), https://www.mathworks.com/help/deeplearning/ug/pretrained-convolutional-neural-networks.html. Manteniendo el tamaño original de la red, se puede hacer transfer learning.
- Se usa ResNet18 de la librería classification_models (https://github.com/qubvel/classification_models)
- Se dejan liberadas las 20 últimas capas, como hace el script de MatLab Coincide con el último cambio de tamaño de las salidas intermedias.
- Se hace validación cruzada con k-fold para $k=5$. En el armado de los datasets de entrenamiento y test, Kather *et al.* usan uno de los $k$ subconjuntos para entrenamiento y los 4 restantes para test. En este código se invierten las cantidades.

In [35]:
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import PIL
import PIL.Image

import pickle
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

from keras.preprocessing.image import ImageDataGenerator  
from keras.models import Model
from keras.layers import Dense

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn import metrics

from classification_models.tfkeras import Classifiers


# Se tuvo que cambiar varios imports en el código de classification_models porque 
# (parece que) apuntaban a una versión viejas de Keras o tensorflow

In [36]:
image_inputPath_str = "C:\\Users\\Usuario\\Desktop\\Real Stuff\\2022\\Practica Supervisada\\Proyecto de Investigacion\\dataset" # parent folder for the data set
image_inputPath = pathlib.Path(image_inputPath_str)

loadPreviousProgress = False # continue where you stopped before
currFn = 'classi3xval' # current filename for saving log files

hyperparam = {}
hyperparam["InitialLearnRate"] = 1e-5      # initial learning rate
hyperparam["ValidationFrequency"] = 150    # check validation performance every N iterations, 500 is 3x per epoch
hyperparam["ValidationPatience"] = 10      # wait N times before abort
hyperparam["L2Regularization"] = 1e-4      # optimization L2 constraint
hyperparam["MiniBatchSize"] = 64           # mini batch size, limited by GPU RAM, default 100 on Titan, 500 on P6000
hyperparam["MaxEpochs"] = 150              # max. epochs for training, default 15
hyperparam["hotLayers"] = 100              # how many layers from the end are not frozen
hyperparam["learnRateFactor"] = 2          # learning rate factor for rewired layers
hyperparam["ExecutionEnvironment"] = 'gpu' # environment for training and classification
hyperparam["PixelRangeShear"] = 5          # max. shear (in pixels) for image augmenter
allHyperparam = list(hyperparam.keys())

In [37]:
# NO ES PARTE DEL SCRIPT
# Chequeo de cantidad de archivos y tamaño de imágenes
for im_class in os.listdir(image_inputPath):
    dir_width = []
    dir_height = []
    file_count = 0
    for im_file in os.listdir(str(image_inputPath) + "\\" + im_class):
        file_count = file_count + 1
        im = PIL.Image.open(str(image_inputPath) + "\\" + im_class + "\\" + im_file)
        width, height = im.size
        dir_width.append(width)
        dir_height.append(height)
    print("Class " + im_class + ": " + str(file_count) + " files, min width: " 
          + str(min(dir_width)) + ", max width: " + str(max(dir_width)) + ", min height: " 
          + str(min(dir_height)) + ", max height: " + str(max(dir_height)))

Class ADIMUC: 3977 files, min width: 512, max width: 512, min height: 512, max height: 512
Class STRMUS: 4000 files, min width: 512, max width: 512, min height: 512, max height: 512
Class TUMSTU: 4000 files, min width: 512, max width: 512, min height: 512, max height: 512


In [38]:
#  READ ALL IMAGES

img_height = 224 # 512
img_width = 224 # 512

allImages = tf.keras.utils.image_dataset_from_directory(
  image_inputPath,
  labels='inferred',
  seed=123,
  image_size=(img_height, img_width),
#   batch_size=hyperparam["MiniBatchSize"]
  batch_size=None, # No hay batch
  label_mode='categorical'
  )
allImages

Found 11977 files belonging to 3 classes.


<ShuffleDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(3,), dtype=tf.float32, name=None))>

In [39]:
# NO ES PARTE DEL SCRIPT

print(type(allImages))
b = allImages.take(10)
print(type(b))
a = list(b.as_numpy_iterator())

print(type(a))
print(len(a))
print(type(a[0]))
print(type(a[0][0]))
print(a[0][0].shape)
print(a[0][1].shape)
print(type(a[0][1]))
print(a[0][1])

<class 'tensorflow.python.data.ops.dataset_ops.ShuffleDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.TakeDataset'>
<class 'list'>
10
<class 'tuple'>
<class 'numpy.ndarray'>
(224, 224, 3)
(3,)
<class 'numpy.ndarray'>
[0. 0. 1.]


In [40]:
# NO ES PARTE DEL SCRIPT
class_names = allImages.class_names

In [41]:
# NO ES PARTE DEL SCRIPT
print(class_names)

['ADIMUC', 'STRMUS', 'TUMSTU']


In [42]:
DATASET_SIZE = allImages.cardinality().numpy()
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

training_set = allImages.take(train_size)
testing_set = allImages.skip(train_size)
validation_set = testing_set.skip(test_size)
testing_set = testing_set.take(test_size)
print('successfully LOADED TRAINING images');

successfully LOADED TRAINING images


In [ ]:
#y = np.concatenate([y for x, y in training_set], axis=0)
#print(y)

In [43]:
hw_factor = hyperparam["PixelRangeShear"]/224
flip_layer = layers.RandomFlip("horizontal_and_vertical")
translation_layer = layers.RandomTranslation(height_factor=hw_factor, width_factor=hw_factor)
resizing_layer = layers.Resizing(img_height, img_width)

training_set_resized = training_set.map(lambda x, y: (resizing_layer(x), y))
testing_set_resized = testing_set.map(lambda x, y: (resizing_layer(x), y))
validation_set_resized = validation_set.map(lambda x, y: (resizing_layer(x), y))


def preprocess(images, labels):
  return preprocess_input(images), labels

training_set_prep = training_set_resized.map(preprocess)
testing_set_prep = testing_set_resized.map(preprocess)
validation_set_prep = validation_set_resized.map(preprocess)


def expand_d(images, labels):
     return tf.expand_dims(images, axis=0), tf.expand_dims(labels, axis=0)

training_set_prep_exp = training_set_prep.map(expand_d)
testing_set_prep_exp = testing_set_prep.map(expand_d)
validation_set_prep_exp = validation_set_prep.map(expand_d)

In [44]:
# THIS AINT NECESSARY
if loadPreviousProgress:
    loaded_model = pickle.load(open("*.mat", 'rb'))
    result = loaded_model.score(X_test, Y_test)
    

In [ ]:
# TRAIN NETWORK

experimentCounter = 0
skipCounter = 0
allModels = {"resnet18"}
for initialLearnRate = [5e-6]:
    hyperparam["InitialLearnRate"] = initialLearnRate
    for hotLayers = [20]:
        hyperparam["HotLayers"] = hotLayers
        for i = 1:
            experimentCounter += 1
            nmodel = allModels{i}

In [47]:
AUTOTUNE = tf.data.AUTOTUNE

start = time.time()
#for idx in range(1, 6):
for idx in range(1, 2):
    print("\n--------------Entrenamiento para k = " + str(idx) + " --------------\n")

    trainSet_cache = training_set.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    testSet_cache = testing_set.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

    trainSet_cache_da = trainSet_cache.map(lambda x, y: (flip_layer(x), y))
    trainSet_cache_da = trainSet_cache_da.map(lambda x, y: (translation_layer(x), y))
    
    ResNet18, preprocess_input = Classifiers.get('resnet18')
    model_orig = ResNet18((224, 224, 3), weights='imagenet')
    new_layer_output = Dense(len(class_names), activation='softmax', name='predictions')
    model = Model(model_orig.input, new_layer_output(model_orig.layers[-3].output))
#     model.summary()
    ldx = 0
    for layer in model.layers:
        ldx = ldx + 1
        if ldx < 69:
            layer.trainable = False
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])

    ep=5
    model.fit(training_set_prep_exp, validation_data=validation_set_prep_exp, epochs=ep, validation_freq=1)
end = time.time()


--------------Entrenamiento para k = 1 --------------

Epoch 1/5
8383/8383 [==============================] - 3587s 427ms/step - loss: 0.2370 - accuracy: 0.9194 - val_loss: 0.2826 - val_accuracy: 0.8949
Epoch 2/5
8383/8383 [==============================] - 3017s 359ms/step - loss: 0.0909 - accuracy: 0.9773 - val_loss: 0.1735 - val_accuracy: 0.9355
Epoch 3/5
8383/8383 [==============================] - 3033s 361ms/step - loss: 0.0545 - accuracy: 0.9858 - val_loss: 0.2187 - val_accuracy: 0.9394
Epoch 4/5
8383/8383 [==============================] - 3488s 415ms/step - loss: 0.0396 - accuracy: 0.9896 - val_loss: 0.4408 - val_accuracy: 0.7681
Epoch 5/5
8383/8383 [==============================] - 2970s 354ms/step - loss: 0.0272 - accuracy: 0.9930 - val_loss: 0.4504 - val_accuracy: 0.8404


In [48]:
# No es parte del script
print(end - start)

16136.873936414719


In [ ]:
SALIDA:

--------------Entrenamiento para k = 1 --------------

Epoch 1/5

2022-03-24 12:52:49.697213: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2022-03-24 12:52:50.994749: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory

9581/9581 [==============================] - 159s 15ms/step - loss: 0.2285 - accuracy: 0.9217 - val_loss: 0.2888 - val_accuracy: 0.8902
Epoch 2/5
9581/9581 [==============================] - 120s 13ms/step - loss: 0.1032 - accuracy: 0.9715 - val_loss: 0.3152 - val_accuracy: 0.8944
Epoch 3/5
9581/9581 [==============================] - 121s 13ms/step - loss: 0.0861 - accuracy: 0.9776 - val_loss: 0.4664 - val_accuracy: 0.7884
Epoch 4/5
9581/9581 [==============================] - 120s 13ms/step - loss: 0.0626 - accuracy: 0.9815 - val_loss: 0.3449 - val_accuracy: 0.8986
Epoch 5/5
9581/9581 [==============================] - 120s 13ms/step - loss: 0.0566 - accuracy: 0.9849 - val_loss: 0.1547 - val_accuracy: 0.9741

--------------Entrenamiento para k = 2 --------------

Epoch 1/5
9581/9581 [==============================] - 115s 11ms/step - loss: 0.2326 - accuracy: 0.9192 - val_loss: 0.5439 - val_accuracy: 0.8264
Epoch 2/5
9581/9581 [==============================] - 86s 9ms/step - loss: 0.0890 - accuracy: 0.9763 - val_loss: 0.2247 - val_accuracy: 0.9265
Epoch 3/5
9581/9581 [==============================] - 85s 9ms/step - loss: 0.0464 - accuracy: 0.9894 - val_loss: 0.3349 - val_accuracy: 0.9011
Epoch 4/5
9581/9581 [==============================] - 85s 9ms/step - loss: 0.0377 - accuracy: 0.9909 - val_loss: 0.2890 - val_accuracy: 0.9240
Epoch 5/5
9581/9581 [==============================] - 85s 9ms/step - loss: 0.0241 - accuracy: 0.9933 - val_loss: 0.4071 - val_accuracy: 0.8756

--------------Entrenamiento para k = 3 --------------

Epoch 1/5
9582/9582 [==============================] - 115s 11ms/step - loss: 0.2544 - accuracy: 0.9092 - val_loss: 3.9616 - val_accuracy: 0.6033
Epoch 2/5
9582/9582 [==============================] - 85s 9ms/step - loss: 0.0737 - accuracy: 0.9795 - val_loss: 0.4025 - val_accuracy: 0.8576
Epoch 3/5
9582/9582 [==============================] - 85s 9ms/step - loss: 0.0499 - accuracy: 0.9863 - val_loss: 0.4569 - val_accuracy: 0.7971
Epoch 4/5
9582/9582 [==============================] - 85s 9ms/step - loss: 0.0357 - accuracy: 0.9912 - val_loss: 1.1441 - val_accuracy: 0.6372
Epoch 5/5
9582/9582 [==============================] - 85s 9ms/step - loss: 0.0248 - accuracy: 0.9927 - val_loss: 0.2920 - val_accuracy: 0.9152

--------------Entrenamiento para k = 4 --------------

Epoch 1/5
9582/9582 [==============================] - 115s 11ms/step - loss: 0.2081 - accuracy: 0.9320 - val_loss: 1.8176 - val_accuracy: 0.4747
Epoch 2/5
9582/9582 [==============================] - 85s 9ms/step - loss: 0.0752 - accuracy: 0.9795 - val_loss: 1.0992 - val_accuracy: 0.7023
Epoch 3/5
9582/9582 [==============================] - 85s 9ms/step - loss: 0.0441 - accuracy: 0.9887 - val_loss: 0.3079 - val_accuracy: 0.8985
Epoch 4/5
9582/9582 [==============================] - 85s 9ms/step - loss: 0.0344 - accuracy: 0.9910 - val_loss: 0.4832 - val_accuracy: 0.8234
Epoch 5/5
9582/9582 [==============================] - 86s 9ms/step - loss: 0.0289 - accuracy: 0.9926 - val_loss: 0.6125 - val_accuracy: 0.7566

--------------Entrenamiento para k = 5 --------------

Epoch 1/5
9582/9582 [==============================] - 115s 11ms/step - loss: 0.2213 - accuracy: 0.9224 - val_loss: 0.3530 - val_accuracy: 0.8894
Epoch 2/5
9582/9582 [==============================] - 86s 9ms/step - loss: 0.0786 - accuracy: 0.9794 - val_loss: 0.2126 - val_accuracy: 0.9165
Epoch 3/5
9582/9582 [==============================] - 85s 9ms/step - loss: 0.0412 - accuracy: 0.9898 - val_loss: 0.6642 - val_accuracy: 0.7937
Epoch 4/5
9582/9582 [==============================] - 85s 9ms/step - loss: 0.0371 - accuracy: 0.9907 - val_loss: 0.4132 - val_accuracy: 0.8501
Epoch 5/5
9582/9582 [==============================] - 86s 9ms/step - loss: 0.0199 - accuracy: 0.9955 - val_loss: 0.1960 - val_accuracy: 0.9286

In [49]:
# Deploy
predictions = model.predict(testing_set_prep_exp)
print(predictions)

1796/1796 [==============================] - 406s 187ms/step
[[1.7060665e-02 9.5512283e-01 2.7816407e-02]
 [2.6109714e-02 8.7888992e-01 9.5000371e-02]
 [9.0290391e-04 1.2468483e-04 9.9897242e-01]
 ...
 [2.8746810e-03 4.3219537e-04 9.9669313e-01]
 [2.6618546e-01 3.9426968e-01 3.3954483e-01]
 [8.8663809e-03 1.9244839e-02 9.7188878e-01]]


In [50]:
# AUC Calculus
y = np.concatenate([y for x, y in testing_set_prep_exp], axis=0)
metrics.roc_auc_score(y, predictions)

0.5095412598503969

In [51]:
# Save Model
filename = "trained_model.sav"
pickle.dump(model, open(filename, "wb"))

INFO:tensorflow:Assets written to: ram://52160456-7064-44d1-897b-aa88bc82d43e/assets


INFO:tensorflow:Assets written to: ram://52160456-7064-44d1-897b-aa88bc82d43e/assets


In [11]:
__________________________________________________________________________________________________
    Layer (type)                            Output Shape               Param #     Connected to                     
   ==================================================================================================
 01  data (InputLayer)                     [(None, 224, 224, 3  0)]                []                               
 02  bn_data (BatchNormalization)           (None, 224, 224, 3)                9   ['data[0][0]']                   
 03  zero_padding2d_18 (ZeroPadding2D)      (None, 230, 230, 3)                0   ['bn_data[0][0]']                
 04  conv0 (Conv2D)                         (None, 112, 112, 64  9408)             ['zero_padding2d_18[0][0]']      
 05  bn0 (BatchNormalization)               (None, 112, 112, 64  256)              ['conv0[0][0]']                  
 06  relu0 (Activation)                     (None, 112, 112, 64  0)                ['bn0[0][0]']                    
 07  zero_padding2d_19 (ZeroPadding2D)      (None, 114, 114, 64  0)                ['relu0[0][0]']                  
 08  pooling0 (MaxPooling2D)                (None, 56, 56, 64)                 0   ['zero_padding2d_19[0][0]']      
 09  stage1_unit1_bn1 (BatchNormalization)  (None, 56, 56, 64)               256   ['pooling0[0][0]']               
 10  stage1_unit1_relu1 (Activation)        (None, 56, 56, 64)                 0   ['stage1_unit1_bn1[0][0]']       
 11  zero_padding2d_20 (ZeroPadding2D)      (None, 58, 58, 64)                 0   ['stage1_unit1_relu1[0][0]']     
 12  stage1_unit1_conv1 (Conv2D)            (None, 56, 56, 64)             36864   ['zero_padding2d_20[0][0]']      
 13  stage1_unit1_bn2 (BatchNormalization)  (None, 56, 56, 64)               256   ['stage1_unit1_conv1[0][0]']     
 14  stage1_unit1_relu2 (Activation)        (None, 56, 56, 64)                 0   ['stage1_unit1_bn2[0][0]']       
 15  zero_padding2d_21 (ZeroPadding2D)      (None, 58, 58, 64)                 0   ['stage1_unit1_relu2[0][0]']     
 16  stage1_unit1_conv2 (Conv2D)            (None, 56, 56, 64)             36864   ['zero_padding2d_21[0][0]']      
 17  stage1_unit1_sc (Conv2D)               (None, 56, 56, 64)              4096   ['stage1_unit1_relu1[0][0]']     
 18  add_8 (Add)                            (None, 56, 56, 64)                 0   ['stage1_unit1_conv2[0][0]', 'stage1_unit1_sc[0][0]']
 19  stage1_unit2_bn1 (BatchNormalization)  (None, 56, 56, 64)               256   ['add_8[0][0]']                  
 20  stage1_unit2_relu1 (Activation)        (None, 56, 56, 64)                 0   ['stage1_unit2_bn1[0][0]']       
 21  zero_padding2d_22 (ZeroPadding2D)      (None, 58, 58, 64)                 0   ['stage1_unit2_relu1[0][0]']     
 22  stage1_unit2_conv1 (Conv2D)            (None, 56, 56, 64)             36864   ['zero_padding2d_22[0][0]']      
 23  stage1_unit2_bn2 (BatchNormalization)  (None, 56, 56, 64)               256   ['stage1_unit2_conv1[0][0]']     
 24  stage1_unit2_relu2 (Activation)        (None, 56, 56, 64)                 0   ['stage1_unit2_bn2[0][0]']       
 25  zero_padding2d_23 (ZeroPadding2D)      (None, 58, 58, 64)                 0   ['stage1_unit2_relu2[0][0]']     
 26  stage1_unit2_conv2 (Conv2D)            (None, 56, 56, 64)             36864   ['zero_padding2d_23[0][0]']      
 27  add_9 (Add)                            (None, 56, 56, 64)                 0   ['stage1_unit2_conv2[0][0]', 'add_8[0][0]']
 28  stage2_unit1_bn1 (BatchNormalization)  (None, 56, 56, 64)               256   ['add_9[0][0]']                  
 29  stage2_unit1_relu1 (Activation)        (None, 56, 56, 64)                 0   ['stage2_unit1_bn1[0][0]']       
 30  zero_padding2d_24 (ZeroPadding2D)      (None, 58, 58, 64)                 0   ['stage2_unit1_relu1[0][0]']     
 31  stage2_unit1_conv1 (Conv2D)            (None, 28, 28, 128)            73728   ['zero_padding2d_24[0][0]']      
 32  stage2_unit1_bn2 (BatchNormalization)  (None, 28, 28, 128)              512   ['stage2_unit1_conv1[0][0]']     
 33  stage2_unit1_relu2 (Activation)        (None, 28, 28, 128)                0   ['stage2_unit1_bn2[0][0]']       
 34  zero_padding2d_25 (ZeroPadding2D)      (None, 30, 30, 128)                0   ['stage2_unit1_relu2[0][0]']     
 35  stage2_unit1_conv2 (Conv2D)            (None, 28, 28, 128)           147456   ['zero_padding2d_25[0][0]']      
 36  stage2_unit1_sc (Conv2D)               (None, 28, 28, 128)             8192   ['stage2_unit1_relu1[0][0]']     
 37  add_10 (Add)                           (None, 28, 28, 128)                0   ['stage2_unit1_conv2[0][0]', 'stage2_unit1_sc[0][0]']
 38  stage2_unit2_bn1 (BatchNormalization)  (None, 28, 28, 128)              512   ['add_10[0][0]']                 
 39  stage2_unit2_relu1 (Activation)        (None, 28, 28, 128)                0   ['stage2_unit2_bn1[0][0]']       
 40  zero_padding2d_26 (ZeroPadding2D)      (None, 30, 30, 128)                0   ['stage2_unit2_relu1[0][0]']     
 41  stage2_unit2_conv1 (Conv2D)            (None, 28, 28, 128)           147456   ['zero_padding2d_26[0][0]']      
 42  stage2_unit2_bn2 (BatchNormalization)  (None, 28, 28, 128)              512   ['stage2_unit2_conv1[0][0]']     
 43  stage2_unit2_relu2 (Activation)        (None, 28, 28, 128)                0   ['stage2_unit2_bn2[0][0]']       
 44  zero_padding2d_27 (ZeroPadding2D)      (None, 30, 30, 128)                0   ['stage2_unit2_relu2[0][0]']     
 45  stage2_unit2_conv2 (Conv2D)            (None, 28, 28, 128)           147456   ['zero_padding2d_27[0][0]']      
 46  add_11 (Add)                           (None, 28, 28, 128)                0   ['stage2_unit2_conv2[0][0]', 'add_10[0][0]']
 47  stage3_unit1_bn1 (BatchNormalization)  (None, 28, 28, 128)              512   ['add_11[0][0]']                 
 48  stage3_unit1_relu1 (Activation)        (None, 28, 28, 128)                0   ['stage3_unit1_bn1[0][0]']       
 49  zero_padding2d_28 (ZeroPadding2D)      (None, 30, 30, 128)                0   ['stage3_unit1_relu1[0][0]']     
 50  stage3_unit1_conv1 (Conv2D)            (None, 14, 14, 256)           294912   ['zero_padding2d_28[0][0]']      
 51  stage3_unit1_bn2 (BatchNormalization)  (None, 14, 14, 256)             1024   ['stage3_unit1_conv1[0][0]']     
 52  stage3_unit1_relu2 (Activation)        (None, 14, 14, 256)                0   ['stage3_unit1_bn2[0][0]']       
 53  zero_padding2d_29 (ZeroPadding2D)      (None, 16, 16, 256)                0   ['stage3_unit1_relu2[0][0]']     
 54  stage3_unit1_conv2 (Conv2D)            (None, 14, 14, 256)           589824   ['zero_padding2d_29[0][0]']      
 55  stage3_unit1_sc (Conv2D)               (None, 14, 14, 256)            32768   ['stage3_unit1_relu1[0][0]']     
 56  add_12 (Add)                           (None, 14, 14, 256)                0   ['stage3_unit1_conv2[0][0]', 'stage3_unit1_sc[0][0]']
 57  stage3_unit2_bn1 (BatchNormalization)  (None, 14, 14, 256)             1024   ['add_12[0][0]']                 
 58  stage3_unit2_relu1 (Activation)        (None, 14, 14, 256)                0   ['stage3_unit2_bn1[0][0]']       
 59  zero_padding2d_30 (ZeroPadding2D)      (None, 16, 16, 256)                0   ['stage3_unit2_relu1[0][0]']     
 60  stage3_unit2_conv1 (Conv2D)            (None, 14, 14, 256)           589824   ['zero_padding2d_30[0][0]']      
 61  stage3_unit2_bn2 (BatchNormalization)  (None, 14, 14, 256)             1024   ['stage3_unit2_conv1[0][0]']     
 62  stage3_unit2_relu2 (Activation)        (None, 14, 14, 256)                0   ['stage3_unit2_bn2[0][0]']       
 63  zero_padding2d_31 (ZeroPadding2D)      (None, 16, 16, 256)                0   ['stage3_unit2_relu2[0][0]']     
 64  stage3_unit2_conv2 (Conv2D)            (None, 14, 14, 256)           589824   ['zero_padding2d_31[0][0]']      
 65  add_13 (Add)                           (None, 14, 14, 256)                0   ['stage3_unit2_conv2[0][0]', 'add_12[0][0]']
 66  stage4_unit1_bn1 (BatchNormalization)  (None, 14, 14, 256)             1024   ['add_13[0][0]']                 
 67  stage4_unit1_relu1 (Activation)        (None, 14, 14, 256)                0   ['stage4_unit1_bn1[0][0]']       
 68  zero_padding2d_32 (ZeroPadding2D)      (None, 16, 16, 256)                0   ['stage4_unit1_relu1[0][0]']     
 69  stage4_unit1_conv1 (Conv2D)            (None, 7, 7, 512)            1179648   ['zero_padding2d_32[0][0]']      
 70  stage4_unit1_bn2 (BatchNormalization)  (None, 7, 7, 512)               2048   ['stage4_unit1_conv1[0][0]']     
 71  stage4_unit1_relu2 (Activation)        (None, 7, 7, 512)                  0   ['stage4_unit1_bn2[0][0]']       
 72  zero_padding2d_33 (ZeroPadding2D)      (None, 9, 9, 512)                  0   ['stage4_unit1_relu2[0][0]']     
 73  stage4_unit1_conv2 (Conv2D)            (None, 7, 7, 512)            2359296   ['zero_padding2d_33[0][0]']      
 74  stage4_unit1_sc (Conv2D)               (None, 7, 7, 512)             131072   ['stage4_unit1_relu1[0][0]']     
 75  add_14 (Add)                           (None, 7, 7, 512)                  0   ['stage4_unit1_conv2[0][0]', 'stage4_unit1_sc[0][0]']
 76  stage4_unit2_bn1 (BatchNormalization)  (None, 7, 7, 512)               2048   ['add_14[0][0]']                 
 77  stage4_unit2_relu1 (Activation)        (None, 7, 7, 512)                  0   ['stage4_unit2_bn1[0][0]']       
 78  zero_padding2d_34 (ZeroPadding2D)      (None, 9, 9, 512)                  0   ['stage4_unit2_relu1[0][0]']     
 79  stage4_unit2_conv1 (Conv2D)            (None, 7, 7, 512)            2359296   ['zero_padding2d_34[0][0]']      
 80  stage4_unit2_bn2 (BatchNormalization)  (None, 7, 7, 512)               2048   ['stage4_unit2_conv1[0][0]']     
 81  stage4_unit2_relu2 (Activation)        (None, 7, 7, 512)                  0   ['stage4_unit2_bn2[0][0]']       
 82  zero_padding2d_35 (ZeroPadding2D)      (None, 9, 9, 512)                  0   ['stage4_unit2_relu2[0][0]']     
 83  stage4_unit2_conv2 (Conv2D)            (None, 7, 7, 512)            2359296   ['zero_padding2d_35[0][0]']      
 84  add_15 (Add)                           (None, 7, 7, 512)                  0   ['stage4_unit2_conv2[0][0]', 'add_14[0][0]']
 85  bn1 (BatchNormalization)               (None, 7, 7, 512)               2048   ['add_15[0][0]']                 
 86  relu1 (Activation)                     (None, 7, 7, 512)                  0   ['bn1[0][0]']                    
 87  pool1 (GlobalAveragePooling2D)         (None, 512)                        0   ['relu1[0][0]']                  
 88  predictions (Dense)                    (None, 3)                       1539   ['pool1[0][0]']  

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 3)